#PySpark
##primeiro passo: instalar e fazer conexão

In [ ]:
 !pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyspark
#conexão
from pyspark.sql import SparkSession
#filtragem 
from pyspark.sql.functions import max, count, col, asc, desc

In [ ]:
#realiza a conexão com o spark
spark = SparkSession.builder\
      .master("local")\
      .appName("DataFrame")\
      .getOrCreate()

##Segundo passo: realizar a leitura e lapidação dos dados

In [ ]:
#guarda o arquivo na base de dados se tornando um dataframe
#obs.: caso seja separado por caracteres, deve ser utilziado sep = (caracter separador)
df = spark.read.csv("/content/imdb_top_1000.csv", header = True, inferSchema = True)

#mostra os tipos de dados de cada coluna
df.printSchema()



root
 |-- Poster_Link: string (nullable = true)
 |-- Series_Title: string (nullable = true)
 |-- Released_Year: string (nullable = true)
 |-- Certificate: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- Overview: string (nullable = true)
 |-- Meta_score: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Star1: string (nullable = true)
 |-- Star2: string (nullable = true)
 |-- Star3: string (nullable = true)
 |-- Star4: string (nullable = true)
 |-- No_of_Votes: string (nullable = true)
 |-- Gross: string (nullable = true)



In [ ]:
#mostra a base de dados
df.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [ ]:
#cria uma variavel que seleciona as colunas da base de dados anterior
df_series = df.select('Series_title','Released_Year','Director','Genre', 'No_of_Votes','Star1','Star2')
df_series.show()

+--------------------+-------------+--------------------+--------------------+-----------+------------------+--------------------+
|        Series_title|Released_Year|            Director|               Genre|No_of_Votes|             Star1|               Star2|
+--------------------+-------------+--------------------+--------------------+-----------+------------------+--------------------+
|The Shawshank Red...|         1994|      Frank Darabont|               Drama|    2343110|       Tim Robbins|      Morgan Freeman|
|       The Godfather|         1972|Francis Ford Coppola|        Crime, Drama|    1620367|     Marlon Brando|           Al Pacino|
|     The Dark Knight|         2008|   Christopher Nolan|Action, Crime, Drama|    2303232|    Christian Bale|        Heath Ledger|
|The Godfather: Pa...|         1974|Francis Ford Coppola|        Crime, Drama|    1129952|         Al Pacino|      Robert De Niro|
|        12 Angry Men|         1957|        Sidney Lumet|        Crime, Drama|     

In [ ]:
#cria uma nova coluna chamada votos como copia da coluna No_of_Votes, mudando sua tipagem para int
df_votos = df_series.withColumn('votos', df_series['No_of_Votes'].cast('int')).drop('No_of_Votes')
#coluna No_of_Votes foi removida com .drop()
df_votos.show()

+--------------------+-------------+--------------------+--------------------+------------------+--------------------+-------+
|        Series_title|Released_Year|            Director|               Genre|             Star1|               Star2|  votos|
+--------------------+-------------+--------------------+--------------------+------------------+--------------------+-------+
|The Shawshank Red...|         1994|      Frank Darabont|               Drama|       Tim Robbins|      Morgan Freeman|2343110|
|       The Godfather|         1972|Francis Ford Coppola|        Crime, Drama|     Marlon Brando|           Al Pacino|1620367|
|     The Dark Knight|         2008|   Christopher Nolan|Action, Crime, Drama|    Christian Bale|        Heath Ledger|2303232|
|The Godfather: Pa...|         1974|Francis Ford Coppola|        Crime, Drama|         Al Pacino|      Robert De Niro|1129952|
|        12 Angry Men|         1957|        Sidney Lumet|        Crime, Drama|       Henry Fonda|         Lee J

#Filtragem

In [ ]:
#demonstração com int
df_votos.filter(df_votos.votos > 1800000).show()

+--------------------+-------------+-----------------+--------------------+-----------------+--------------------+-------+
|        Series_title|Released_Year|         Director|               Genre|            Star1|               Star2|  votos|
+--------------------+-------------+-----------------+--------------------+-----------------+--------------------+-------+
|The Shawshank Red...|         1994|   Frank Darabont|               Drama|      Tim Robbins|      Morgan Freeman|2343110|
|     The Dark Knight|         2008|Christopher Nolan|Action, Crime, Drama|   Christian Bale|        Heath Ledger|2303232|
|        Pulp Fiction|         1994|Quentin Tarantino|        Crime, Drama|    John Travolta|         Uma Thurman|1826188|
|           Inception|         2010|Christopher Nolan|Action, Adventure...|Leonardo DiCaprio|Joseph Gordon-Levitt|2067042|
|          Fight Club|         1999|    David Fincher|               Drama|        Brad Pitt|       Edward Norton|1854740|
|        Forrest

In [ ]:
#demonstração com string
df_votos.filter(df_votos.Star1 == 'Joaquin Phoenix').show()

+-------------+-------------+-------------+--------------------+---------------+-----------------+------+
| Series_title|Released_Year|     Director|               Genre|          Star1|            Star2| votos|
+-------------+-------------+-------------+--------------------+---------------+-----------------+------+
|        Joker|         2019|Todd Phillips|Crime, Drama, Thr...|Joaquin Phoenix|   Robert De Niro|939252|
|          Her|         2013|  Spike Jonze|Drama, Romance, S...|Joaquin Phoenix|        Amy Adams|540772|
|Walk the Line|         2005|James Mangold|Biography, Drama,...|Joaquin Phoenix|Reese Witherspoon|234207|
+-------------+-------------+-------------+--------------------+---------------+-----------------+------+



In [ ]:
#para pegar o maximo
df_max = df_votos.agg(max('votos').alias('max_votos'))
df_max.show()

+---------+
|max_votos|
+---------+
|  2343110|
+---------+



In [ ]:
# realiza a filtragem de linhas que tenham o numero de votos igual  a 2343110
df_mega =  df_votos.filter(df_votos.votos ==  2343110).show()

+--------------------+-------------+--------------+-----+-----------+--------------+-------+
|        Series_title|Released_Year|      Director|Genre|      Star1|         Star2|  votos|
+--------------------+-------------+--------------+-----+-----------+--------------+-------+
|The Shawshank Red...|         1994|Frank Darabont|Drama|Tim Robbins|Morgan Freeman|2343110|
+--------------------+-------------+--------------+-----+-----------+--------------+-------+



In [ ]:
#realizar a contagem 
#contagem de filmes por Estrelas
df_sum = df_votos.groupBy('Star1').count()
df_sum.show()

+-----------------+-----+
|            Star1|count|
+-----------------+-----+
| Daniel Day-Lewis|    4|
|   Noémie Merlant|    1|
| Laurence Olivier|    2|
|    Seung-Yun Lee|    1|
|Mikael Persbrandt|    1|
|     Robert Donat|    1|
|    John Travolta|    1|
|      Kim Min-hee|    1|
|     Silvia Pinal|    1|
|       Emma Stone|    1|
|     Glen Hansard|    1|
|   Ingrid Bergman|    1|
|     Emile Hirsch|    1|
|       Barry Cook|    1|
|         Tim Roth|    1|
|        Amit Sadh|    1|
|     Eileen Walsh|    1|
|   Peter Dinklage|    1|
|Emilio Echevarría|    1|
|    Griffin Dunne|    1|
+-----------------+-----+
only showing top 20 rows



In [ ]:
#mesma contagem só que em ordem decrescente 
df_sum.orderBy(col('count').desc()).show(10)

+-----------------+-----+
|            Star1|count|
+-----------------+-----+
|        Tom Hanks|   11|
|   Robert De Niro|   11|
|        Al Pacino|    9|
|   Clint Eastwood|    9|
|Leonardo DiCaprio|    9|
|  Humphrey Bogart|    9|
|   Christian Bale|    8|
|    James Stewart|    8|
|   Toshirô Mifune|    7|
|Denzel Washington|    7|
+-----------------+-----+
only showing top 10 rows



In [ ]:
#selecionando filmes que contem Joaquin Phoenix' e'Christian Bale' mostrando apenas titulo e votos com o nome
df_votos.select('Star1','Star2','Series_title','votos')\
  .filter(df_votos.Star1.isin('Joaquin Phoenix','Christian Bale','Chris Pratt')).show()

+---------------+--------------------+--------------------+-------+
|          Star1|               Star2|        Series_title|  votos|
+---------------+--------------------+--------------------+-------+
| Christian Bale|        Heath Ledger|     The Dark Knight|2303232|
|Joaquin Phoenix|      Robert De Niro|               Joker| 939252|
| Christian Bale|        Hugh Jackman|        The Prestige|1190259|
| Christian Bale|           Tom Hardy|The Dark Knight R...|1516346|
| Christian Bale|       Michael Caine|       Batman Begins|1308302|
|    Chris Pratt|          Vin Diesel|Guardians of the ...|1043455|
|Joaquin Phoenix|           Amy Adams|                 Her| 540772|
| Christian Bale|        Steve Carell|       The Big Short| 362942|
|Joaquin Phoenix|   Reese Witherspoon|       Walk the Line| 234207|
| Christian Bale|Jennifer Jason Leigh|       The Machinist| 358432|
| Christian Bale|      John Malkovich|   Empire of the Sun| 115677|
|    Chris Pratt|         Zoe Saldana|Guardians 

In [ ]:
#filmes que não contem 'Joaquin Phoenix' e 'Christian Bale'
df_votos.select('Star1','Series_title','votos')\
  .filter(~df_votos.Star1.isin('Joaquin Phoenix','Christian Bale', )).show()

+------------------+--------------------+-------+
|             Star1|        Series_title|  votos|
+------------------+--------------------+-------+
|       Tim Robbins|The Shawshank Red...|2343110|
|     Marlon Brando|       The Godfather|1620367|
|         Al Pacino|The Godfather: Pa...|1129952|
|       Henry Fonda|        12 Angry Men| 689845|
|       Elijah Wood|The Lord of the R...|1642758|
|     John Travolta|        Pulp Fiction|1826188|
|       Liam Neeson|    Schindler's List|1213505|
| Leonardo DiCaprio|           Inception|2067042|
|         Brad Pitt|          Fight Club|1854740|
|       Elijah Wood|The Lord of the R...|1661481|
|         Tom Hanks|        Forrest Gump|1809221|
|    Clint Eastwood|Il buono, il brut...| 688390|
|       Elijah Wood|The Lord of the R...|1485555|
|   Lilly Wachowski|          The Matrix|1676426|
|    Robert De Niro|          Goodfellas|1020727|
|       Mark Hamill|Star Wars: Episod...|1159315|
|    Jack Nicholson|One Flew Over the...| 918088|


###Filtragem com SQL

In [ ]:
df_votos.createOrReplaceTempView('movies')
spark.sql('SELECT Star1, count(*) as qtd from movies GROUP BY Star1 ORDER BY qtd desc').show(truncate = False)

+-----------------+---+
|Star1            |qtd|
+-----------------+---+
|Tom Hanks        |11 |
|Robert De Niro   |11 |
|Leonardo DiCaprio|9  |
|Clint Eastwood   |9  |
|Al Pacino        |9  |
|Humphrey Bogart  |9  |
|James Stewart    |8  |
|Christian Bale   |8  |
|Johnny Depp      |7  |
|Denzel Washington|7  |
|Toshirô Mifune   |7  |
|Aamir Khan       |7  |
|Jake Gyllenhaal  |6  |
|Daniel Radcliffe |6  |
|Cary Grant       |6  |
|Charles Chaplin  |6  |
|Tom Cruise       |6  |
|Russell Crowe    |5  |
|Bruce Willis     |5  |
|Ethan Coen       |5  |
+-----------------+---+
only showing top 20 rows



Join

In [ ]:
#importando a nova base de dados
df_reviews = spark.read.csv("/content/IMDB-Movie-Data.csv",header = True, inferSchema = True)
df_reviews.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|   

In [ ]:
#analizando suas colunas
df_reviews.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [ ]:
#revendo os dados do df anterior
df.show()

+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|         Poster_Link|        Series_Title|Released_Year|Certificate|Runtime|               Genre|IMDB_Rating|            Overview|Meta_score|            Director|             Star1|               Star2|             Star3|               Star4|No_of_Votes|      Gross|
+--------------------+--------------------+-------------+-----------+-------+--------------------+-----------+--------------------+----------+--------------------+------------------+--------------------+------------------+--------------------+-----------+-----------+
|https://m.media-a...|The Shawshank Red...|         1994|          A|142 min|               Drama|        9.3|Two imprisoned me...|        80|      Frank Darabont|       Tim Robbins|      Morgan F

In [ ]:
#utilizando Joins
#obs.: não consegui achar outra base de dados para fazer o join
df_Junto = df.join(df_reviews, df.Series_Title == df_reviews.Title, 'inner')\
      .select(df_reviews.Rank, df.Series_Title, df_reviews.Year, df.Star1, df.Star2,df.Meta_score, df_reviews.Rating)
# criada uma base que é a junção das duas bases que utiliza as colunas das duas

In [ ]:
#fazendo uma consulta na nova base criada
print("Melhores filmes por Rating")
df_Junto.filter(df_Junto.Meta_score >= 60)\
        .orderBy(df_Junto.Rating.desc())\
        .show(10)

Melhores filmes por Rating
+----+--------------------+----+-------------------+--------------------+----------+------+
|Rank|        Series_Title|Year|              Star1|               Star2|Meta_score|Rating|
+----+--------------------+----+-------------------+--------------------+----------+------+
|  55|     The Dark Knight|2008|     Christian Bale|        Heath Ledger|        84|     9|
|  81|           Inception|2010|  Leonardo DiCaprio|Joseph Gordon-Levitt|        74|   8.8|
|  37|        Interstellar|2014|Matthew McConaughey|       Anne Hathaway|        74|   8.6|
| 100|        The Departed|2006|  Leonardo DiCaprio|          Matt Damon|        85|   8.5|
|  65|        The Prestige|2006|     Christian Bale|        Hugh Jackman|        66|   8.5|
| 125|The Dark Knight R...|2012|     Christian Bale|           Tom Hardy|        78|   8.5|
| 477| The Lives of Others|2006|        Ulrich Mühe|      Martina Gedeck|        89|   8.5|
| 134|            Whiplash|2014|       Miles Teller| 